### Bibliotecas

In [1]:
import pandas as pd
import numpy as np
import os
import pickle
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

C:\Users\1010h\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Obtener PKL's

### Modelos

In [1]:
# Diccionario de modelos
MODELOS = {
    "sentence_similarity": 'hiiamsid/sentence_similarity_spanish_es',
    "sentence-transformers 1": 'uzairnaeem/finetuned_sentence_similarity_spanish_es',
    "sentence-transformers 2": "mrm8488/modernbert-embed-base-ft-sts-spanish-matryoshka-768-64-5e",
    "sentence-transformers 3" : 'Santp98/SBERT-pairs-bert-base-spanish-wwm-cased'
}

In [ ]:
def cargar_modelo(nombre_modelo):
    print(f"Cargando modelo: {nombre_modelo}")
    modelo = SentenceTransformer(nombre_modelo)
    return modelo

def obtener_embeddings(df, columna, modelo):
    textos = df[columna].astype(str).tolist()
    print(f"Obteniendo embeddings para la columna '{columna}'...")
    embeddings_obtenidos = modelo.encode(textos, convert_to_tensor=False)
    # .encode: convierte el texto (textos) en vectores numéricos (embeddings), esto solo funciona con SentenceTransformer
    # convert_to_tensor=False indica que el resultado se devolvera como un array de numpy en lugar de un tensor de pytorch
    return embeddings_obtenidos

def guardar_embeddings(embeddings, nombre_archivo):
    with open(nombre_archivo, "wb") as f:
        # usamos write binary para almacenar los pkl's
        pickle.dump(embeddings, f)
        # con dump guardamos los embeddings en el archivo que llamamos "f"
    print(f"Embeddings guardados en {nombre_archivo}")
    
def main():
    try:
        df = pd.read_csv("protocolos_completo_limpios.csv")
    except Exception as e:
        print("Error al leer el archivo.", e)
        return

    # campo a comparar
    campo = input("Selecciona el campo (Titulo, resumen, objetivos, claves) para obtener sus PKL's: ").strip()
    # si se ingresa mal escrito el campo o no está en el df, entonces:
    if campo not in df.columns:
        print("El campo ingresado no se encuentra en los datos.")
        return

    # ruta para guardar los pkl's
    ruta_destino = "C:/Users/1010h/Desktop/Embeddings/Embeddings/PKL"

    # iterar sobre cada modelo para calcular y guardar los embeddings
    for clave_modelo, nombre_modelo in MODELOS.items():
        modelo = cargar_modelo(nombre_modelo)
        embeddings_obtenidos = obtener_embeddings(df, campo, modelo)
        archivo_embeddings = os.path.join(ruta_destino, f"{campo}_{clave_modelo}_embeddings.pkl")
        guardar_embeddings(embeddings_obtenidos, archivo_embeddings)

if __name__ == "__main__":
    main()

############
# top modelos:
# EL mejor es el 3, luego el 1, 2 y al ultimo el 4

# Comparar campos

In [ ]:
CAMPOS = ["Titulo", "resumen", "objetivos", "claves"]

# preguntar al usuario cual campo quiere comparar
while True:
    campo = input(f"Selecciona un campo de los siguientes {CAMPOS} para comparar: ").strip()
    if campo in CAMPOS:
        break
    # validamos que el campo exista en CAMPOS
    print(f"El campo '{campo}' no es válido. Debe ser uno de: {CAMPOS}")
    exit()

print("Campo seleccionado: ", campo)
df = pd.read_csv("protocolos_completo_limpios.csv")

while True:
    # query por comparar
    query = input("Ingresa el texto para comparar: ").strip()
    print("\nConsulta: ", query)

    # procesar cada modelo con su respectivo archivo pkl
    for clave_modelo, nombre_modelo in MODELOS.items():
        print(f"\n===== Modelo: {clave_modelo} =====")
        modelo = SentenceTransformer(nombre_modelo)
        # construimos la ruta del archivo pkl correspondiente al campo y modelo para que lo encuentre
        nombre_pkl = f"{campo}_{clave_modelo}_embeddings.pkl"
        ruta_pkl = os.path.join("PKL", nombre_pkl)

        # Verificar que el archivo pkl exista
        if not os.path.exists(ruta_pkl):
            print(f"No se encontró el archivo pkl para este campo y modelo: {ruta_pkl}")
            continue

        # carga embeddings desde los pkl's
        with open(ruta_pkl, "rb") as f:
            embeddings_cargados = pickle.load(f)

        # Generar el embedding para la consulta
        embedding_query = modelo.encode(query, convert_to_tensor=False)
        # convert_to_tensor=False indica que el resultado se devolvera como un array de numpy en lugar de un tensor de pytorch,
        # esto para calcular la similitud coseno despues

        # aqui se obtiene la similitud coseno entre la query y cada embedding de los pkl
        simi_coseno = cosine_similarity([embedding_query], embeddings_cargados)[0]
        df_temp = df.copy()
        df_temp["similitud"] = simi_coseno
        df_temp = df_temp.sort_values(by="similitud", ascending=False).head(20) # mostrar solo las primeras 20 coincidencias

        # limitamos la info de la columna seleccionada a 190 caracteres
        df_temp[campo] = df_temp[campo].apply(lambda x: x[:190] + "..." if isinstance(x, str) and len(x) > 190 else x)

        print(f"\nResultados de similitud para la consulta '{query}' con el modelo '{clave_modelo}':")
        print(df_temp[["TT", campo, "similitud"]].to_string(index=False))

Campo seleccionado:  Titulo

Consulta:  Prototipo de Sistema de ReconstrucciÃ³n Tridimensional de un Espacio Cerrado

===== Modelo: sentence_similarity =====

Resultados de similitud para la consulta 'Prototipo de Sistema de ReconstrucciÃ³n Tridimensional de un Espacio Cerrado' con el modelo 'sentence_similarity':
       TT                                                                                                                                                                 Titulo  similitud
2016-A044                                                                                           Prototipo de Sistema de ReconstrucciÃ³n Tridimensional de un Espacio Cerrado   1.000000
23-2-0015                                                                                                                 Prototipo de CreaciÃ³n de MÃºsica con Redes Neuronales   0.513818
2019-B008                                                                                       Sistema clasificador de 